In [1]:
import os, glob, shutil, argparse, re, copy
import pandas as pd
from pathlib import Path
import numpy as np
import training_info

logdir=training_info.logdir
dir_pattern = logdir+'/{}/{}/{}/{}/epoch{}/b{}/worker{}/thread{}/'
aug_pattern = logdir+'/{}/{}/'
simple_dir_pattern = logdir+'/{}'
logtypes=training_info.logtypes
datasets=training_info.datasets
models=training_info.models
term=training_info.term
output_file="output.log"

ModuleNotFoundError: No module named 'training_info'

In [ ]:
def get_dirname():
    dir_names = {}
    for logtype in logtypes:
        dir_names[logtype]=glob.glob(simple_dir_pattern.format(logtype)+f"/*/*/*/*/*/*/*/")

    return dir_names

def replace_str(target):
    target=target.replace('\n', '')
    target=target.replace(',', '')
    return target

def find_value(arr, target, jumpto=1):
    try:
        num=replace_str(arr[arr.index(target)+jumpto])
    except:
        print(arr, target, arr[arr.index(target)+jumpto])
        num='NA'
    return num

parsed_dir=training_info.parsed_dir + "/ds"
os.makedirs(parsed_dir, exist_ok=True)

breakdown_col_name=["Time","Index queues","Worker result queue","Data queue","Consumed"]

dir_names=get_dirname()
print(dir_names)

{'train_only': ['./log/DDP/train_only/1024batch10/default/resnet18/epoch5/b1024/worker4/thread0/', './log/DDP/train_only/1024batch/default/resnet18/epoch5/b1024/worker2/thread0/', './log/DDP/train_only/1024batch/default/resnet18/epoch5/b1024/worker24/thread0/', './log/DDP/train_only/1024batch/default/resnet18/epoch5/b256/worker24/thread0/'], 'loadNtrain': ['./log/DDP/loadNtrain/size5/default/resnet18/epoch5/b256/worker2/thread0/', './log/DDP/loadNtrain/1024batch10/default/resnet18/epoch5/b1024/worker2/thread0/', './log/DDP/loadNtrain/1024batch10/default/resnet18/epoch5/b1024/worker24/thread0/', './log/DDP/loadNtrain/1024batch10/default/resnet18/epoch5/b256/worker24/thread0/', './log/DDP/loadNtrain/1024batch/default/resnet18/epoch5/b1024/worker2/thread0/'], 'prepNloadNtrain': ['./log/DDP/prepNloadNtrain/size5/default/resnet18/epoch5/b1024/worker4/thread0/', './log/DDP/prepNloadNtrain/size5/default/resnet18/epoch5/b1024/worker16/thread0/', './log/DDP/prepNloadNtrain/size5/default/resnet1

In [ ]:
def parser(dir_names, datasets):
    index_queue_put_pattern = re.compile("PUT .* worker_queue_idx")
    index_queue_get_pattern = re.compile("worker_id: [0-9]+, GET")
    worker_result_queue_put_pattern = re.compile("worker_id: [0-9]+, PUT")
    worker_result_queue_get_pattern = re.compile("pin_memory GET .* _worker_result_queue")
    data_queue_put_pattern = re.compile("pin_memory PUT .* data_queue")
    data_queue_get_pattern = re.compile("GET data object at .* _data_queue")
    consumption_pattern = re.compile("Finish iteration[0-9]+")
    
    for logtype in logtypes:
        for logdir in dir_names[logtype]:
            if not os.path.exists(logdir):
                    continue
            logdir_list=logdir.split('/')
            
            dataset=find_value(logdir_list, logtype)
            aug=find_value(logdir_list, dataset)
            model=find_value(logdir_list, aug)
            epoch=find_value(logdir_list, model)
            batchsize=find_value(logdir_list, epoch)
            worker=find_value(logdir_list, batchsize)
            
            worker_num = int(worker.replace("worker",""))
            parse_filename=f"{logtype}_{dataset}_{model}_{aug}_{worker}_{epoch}_{batchsize}_ds"
            
            logfile=logdir +"/"+ output_file
            breakdown_parsed_log = []
            weird_idx_log=[]
            weird_idx_info=[0]
            data_log=[0,0,0,0,0]
            worker_name = []
            weird_idx_name=["Time"]
            
            for i in range(worker_num):
                data_log.append(0)
                worker_name.append(f"Worker{i} index queue")
            
            for line in open(logfile, 'r').readlines():
                if re.search(index_queue_put_pattern, line) is not None: # start log
                    replace_txt=line.replace('\t',' ')
                    test =replace_txt.split(' ')
                    info = list(filter(lambda x: x != "", test))
                    
                    data_log[0] = find_value(info, "DEBUG:", -1) # Time
                    data_log[1] += 1
                    worker_queue = find_value(info, "_index_queues:\n", -1)
                    worker_queue_num = int(worker_queue.replace("worker_queue_idx",""))
                    data_log[5+worker_queue_num] += 1
                    
                    breakdown_parsed_log.append(data_log)
                    data_log=copy.deepcopy(data_log)
                                        
                    not_processed_batch_num = find_value(info, "PUT")
                    not_processed_batch_num=not_processed_batch_num.replace("(","")
                    batch_num=int(not_processed_batch_num.replace(",",""))
                    if batch_num >= len(weird_idx_info)-1:
                        weird_idx_name.append(f"Batch Idx {batch_num}")
                        weird_idx_info.append(0)
                    
                elif re.search(index_queue_get_pattern, line) is not None: # start log
                    replace_txt=line.replace('\t',' ')
                    test =replace_txt.split(' ')
                    info = list(filter(lambda x: x != "", test))
                                        
                    data_log[0] = find_value(info, "DEBUG:", -1) # Time
                    data_log[1] -= 1
                    worker_queue = find_value(info, "worker_id:")
                    worker_queue_num = int(worker_queue.replace(",",""))
                    data_log[5+worker_queue_num] -= 1
                    
                    breakdown_parsed_log.append(data_log)
                    data_log=copy.deepcopy(data_log)
                elif re.search(worker_result_queue_put_pattern, line) is not None: # start log
                    replace_txt=line.replace('\t',' ')
                    test =replace_txt.split(' ')
                    info = list(filter(lambda x: x != "", test))
                    
                    data_log[0] = find_value(info, "DEBUG:", -1) # Time
                    data_log[2] += 1

                    breakdown_parsed_log.append(data_log)
                    data_log=copy.deepcopy(data_log)
                elif re.search(worker_result_queue_get_pattern, line) is not None: # start log
                    replace_txt=line.replace('\t',' ')
                    test =replace_txt.split(' ')
                    info = list(filter(lambda x: x != "", test))
                    
                    data_log[0] = find_value(info, "DEBUG:", -1) # Time
                    data_log[2] -= 1

                    breakdown_parsed_log.append(data_log)
                    data_log=copy.deepcopy(data_log)
                    
                elif re.search(data_queue_put_pattern, line) is not None: # start log
                    replace_txt=line.replace('\t',' ')
                    test =replace_txt.split(' ')
                    info = list(filter(lambda x: x != "", test))
                    
                    data_log[0] = find_value(info, "DEBUG:", -1) # Time
                    weird_idx_info[0] = data_log[0]
                    data_log[3] += 1

                    breakdown_parsed_log.append(data_log)
                    data_log=copy.deepcopy(data_log)
                    
                    not_processed_batch_num = find_value(info, "PUT")
                    not_processed_batch_num=not_processed_batch_num.replace("(","")
                    batch_num=int(not_processed_batch_num.replace(",",""))
                    try:
                        weird_idx_info[1+batch_num]+=1
                    except:
                        print(batch_num)
                        print(weird_idx_info)
                        print(weird_idx_log)
                        return
                    
                    weird_idx_log.append(weird_idx_info)
                    weird_idx_info=copy.deepcopy(weird_idx_info)
                elif re.search(data_queue_get_pattern, line) is not None: # start log
                    replace_txt=line.replace('\t',' ')
                    test =replace_txt.split(' ')
                    info = list(filter(lambda x: x != "", test))
                    
                    data_log[0] = find_value(info, "DEBUG:", -1) # Time
                    data_log[3] -= 1

                    breakdown_parsed_log.append(data_log)
                    data_log=copy.deepcopy(data_log)
                elif re.search(consumption_pattern, line): # start log
                    replace_txt=line.replace('\t',' ')
                    test =replace_txt.split(' ')
                    info = list(filter(lambda x: x != "", test))
                    
                    data_log[0] = find_value(info, "DEBUG:", -1) # Time
                    data_log[4] += 1

                    breakdown_parsed_log.append(data_log)
                    data_log=copy.deepcopy(data_log)
                else:
                    pass
            
            current_col_name = breakdown_col_name +worker_name
            breakdown_df = pd.DataFrame(breakdown_parsed_log,
                              columns=current_col_name)
            breakdown_df.to_csv(parsed_dir+"/"+parse_filename+".csv", sep=',', na_rep='NA')
#             print(weird_idx_name)
            weird_idx_df = pd.DataFrame(weird_idx_log,
                              columns=weird_idx_name)
            weird_idx_df.to_csv(parsed_dir+"/"+parse_filename+"WeirdIdx.csv", sep=',', na_rep=0)

In [ ]:
parser(dir_names, datasets)

KeyboardInterrupt: 